### Deep drive

In [8]:
from langchain_openai import ChatOpenAI
from constants import OPENAI_KEY
import os
os.environ["OPENAI_API_KEY"] = OPENAI_KEY
llm = ChatOpenAI(openai_api_key=OPENAI_KEY, temperature=0.7)

In [6]:
from langchain.prompts import ChatPromptTemplate

In [9]:
# First we need a prompt that we can pass into an LLM to generate this search query
from langchain_openai import OpenAIEmbeddings
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
import faiss
from langchain.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS


embedding_size = 1536 # Dimensions of the OpenAIEmbeddings
index = faiss.IndexFlatL2(embedding_size)
embedding_fn = OpenAIEmbeddings().embed_query
vectorstore = FAISS(embedding_fn, index, InMemoryDocstore({}), {})

retriever = vectorstore.as_retriever(search_kwargs=dict(k=1))

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


In [10]:
retriever_chain

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['FAISS'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001B6DA58B0D0>, search_kwargs={'k': 1}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Given the above conversation, generate a search query to look up in order

In [11]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

KeyError: 0

In [12]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI bot. Your name is {name}."),
    ("human", "Hello, how are you doing?"),
    ("ai", "I'm doing well, thanks!"),
    ("human", "{user_input}"),
])

messages = template.format_messages(
    name="Bob",
    user_input="What is your name?"
)

In [13]:
messages

[SystemMessage(content='You are a helpful AI bot. Your name is Bob.'),
 HumanMessage(content='Hello, how are you doing?'),
 AIMessage(content="I'm doing well, thanks!"),
 HumanMessage(content='What is your name?')]

In [16]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

In [17]:
template = ChatPromptTemplate.from_messages([
    SystemMessage(content="hello"),
    ("human", "Hello, how are you?"),
])

### deep drive


In [19]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.agents.types import AgentType
from langchain.agents import initialize_agent
from langchain.tools import Tool
from langchain.prompts import MessagesPlaceholder


llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)

conversational_memory = ConversationBufferWindowMemory(
    memory_key="chat_history", return_messages=True
)

retriever = vectorstore.as_retriever(search_kwargs=dict(k=1))

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
)

In [20]:
system_message = """
        You are a Virtual Vet. "
        "You should help clients with their concerns about their pets and provide helpful solutions."
        "You can ask questions to help you understand and diagnose the problem."
        "You should only talk within the context of problem."
        "If you are unsure of how to help, you can suggest the client to go to the nearest clink of their place."
        "You should talk on German, unless the client talks in English."
        """
tools = [
    Tool(
        name="qa-vet",
        func=qa.run,
        description="Useful when you need to answer vet questions",
    )
]
executor = initialize_agent(
    agent = AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    memory=conversational_memory,
    agent_kwargs={"system_message": system_message},
    verbose=True,
)

d:\youtube\langchain\envlang\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [31]:
executor.agent

ConversationalChatAgent(llm_chain=LLMChain(prompt=ChatPromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='\n        You are a Virtual Vet. "\n        "You should help clients with their concerns about their pets and provide helpful solutions."\n        "Yo

In [32]:
executor.run(prompt)

d:\youtube\langchain\envlang\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
{"action": "qa-vet", "action_input": "Ich habe ein Problem mit meinem Haustier. Es scheint Schmerzen zu haben und ich weiß nicht, was ich tun soll."}

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-vHxCP***************************************wuXT. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}